In [ ]:
import sys
import pathlib
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))

import numpy as np
import matplotlib.pyplot as plt

import networkx as nx
import dwave_networkx as dnx
import minorminer

import dwave.system
from dwave.system import DWaveSampler, EmbeddingComposite

from src.particle_funcs import distance_matrix as distance_matrix
import src.leap_funcs.qubo.q_matrix as q_matrix

In [ ]:
num_particles = 15

part_coords_n = np.zeros((num_particles,2))
part_coords_nm1 = np.zeros((num_particles,2))

for i in range(np.shape(part_coords_n)[0]):
    part_coords_n[i,:] = [0, i]
    part_coords_nm1[i,:] = [0.5*1, i]

fig_initial, axs_initial = plt.subplots(1,1)
axs_initial.scatter(part_coords_n[:,0], part_coords_n[:,1], label="n")
axs_initial.scatter(part_coords_nm1[:,0], part_coords_nm1[:,1], label="n-1")
fig_initial.legend()
fig_initial.show()

correct_sol = np.zeros(num_particles*num_particles)
for i in range(1, num_particles+1):
    correct_sol[(i-1)*num_particles + i -1] = 1.

print(correct_sol)

In [ ]:
distances = distance_matrix.calc_phi_ij(part_coords_n, part_coords_nm1)
print(distances)
print(np.max(distances))

In [ ]:
Q_dist = q_matrix.q_dist(distances)
assert (Q_dist.transpose() == Q_dist).all(), 'Q_dist is not symmetric'
Q_part = q_matrix.q_part(np.shape(distances)[0])
Q_pos = q_matrix.q_pos(np.shape(distances)[0])
Q_array = Q_dist + Q_part + Q_pos
with np.printoptions(precision=3):
    print(Q_dist)
    print(Q_part)
    print(Q_pos)
    print(Q_pos[0,:])
    print(Q_array)

Q_dict = q_matrix.Q_convert_to_dict(Q_array)
print(Q_dict)

In [ ]:
with open('../API_Token.txt') as file:
    token = file.readline().rstrip()
    architecture = file.readline().rstrip()

sampler = DWaveSampler(token=token , architecture='pegasus')
sampler_graph = sampler.to_networkx_graph()

In [ ]:
#%matplotlib widget
#f, axes = plt.subplots(1, 1)
#plt.ion()
#dnx.draw_pegasus(sampler_graph, with_labels=True, crosses=True, node_color="Yellow", ax=axes)
#plt.show()

In [ ]:
#embedded_G = minorminer.find_embedding(Q_dict,sampler_graph, interactive=True, verbose=3, max_no_improvement=100, tries=50, chainlength_patience=50)
#embedded_G = minorminer.find_embedding(Q_dict,sampler_graph, interactive=True, verbose=3, max_no_improvement=100, tries=200, chainlength_patience=100, threads=6)
embedded_G = minorminer.find_embedding(Q_dict,sampler_graph, interactive=True, verbose=3, max_no_improvement=1000, tries=1000, chainlength_patience=100, threads=6)
#embedded_G = minorminer.find_embedding(Q_dict,sampler_graph, interactive=True, verbose=3)

In [ ]:
f, axes = plt.subplots(1, 1)
dnx.draw_pegasus_embedding(sampler_graph, embedded_G, with_labels=True, crosses=True, ax=axes)
#dnx.draw_pegasus(H, with_labels=True, crosses=True, node_color="Yellow", ax=axes)
#dnx.draw_pegasus(embedded_G, crosses=True, node_color='b', style='dashed',
#          edge_color='b', width=3, ax=axes)
plt.show()


In [ ]:
figsize_man = (75,75)
dpi_man = None
plt.ioff()
f, axes = plt.subplots(1, 1, figsize=figsize_man)
dnx.draw_pegasus_embedding(sampler_graph, embedded_G, with_labels=True, crosses=True, ax=axes)
plt.savefig('embedding.png')